In [2]:
from googlesearch import search

In [6]:
query = "kojima"
results_list = []
for i in search(query,        # The query you want to run
                tld = 'com',  # The top level domain
                lang = 'en',  # The language
                num = 10,     # Number of results per page
                start = 0,    # First result to retrieve
                stop = 10,  # Last result to retrieve
                pause = 2.0,  # Lapse between HTTP requests
               ):
    results_list.append(i)
    print(i)

http://t3.gstatic.com/licensed-image?q=tbn:ANd9GcTHHZI3_Bhpn16_rz9lQJKbpFFPbxbEdHF6w2A_NCMWVoMstdw4vaeTclm_gRg5
https://en.wikipedia.org/wiki/Hideo_Kojima
https://en.wikipedia.org/wiki/Policenauts
https://en.wikipedia.org/wiki/Snatcher_(video_game)
https://en.wikipedia.org/wiki/Zone_of_the_Enders
https://en.wikipedia.org/wiki/Penguin_Adventure
https://www.videogameschronicle.com/news/hideo-kojimas-self-made-death-standing-directors-cut-trailer-shows-off-its-new-wide-mode/
https://www.gamepur.com/news/death-stranding-directors-cut-final-trailer-edited-by-hideo-kojima-shows-widescreen-mode
https://www.rockpapershotgun.com/hideo-kojima-is-a-luxury-brand
https://en.wikipedia.org/wiki/Satoshi_Kojima


In [22]:
from bs4 import BeautifulSoup
import requests

In [25]:

html_page = requests.get(results_list[0])

soup = BeautifulSoup(html_page.text, "html.parser")

urls = []
for link in soup.find_all('a'):
    print(link.get('href'))
print(urls)


None
#mw-head
#searchInput
/wiki/File:Hideo_Kojima_-_Tokyo_Game_Show_2011_(1)_(cropped).jpg
/wiki/Tokyo_Game_Show
#cite_note-ign_bio-1
/wiki/Setagaya
/wiki/Video_game_designer
/wiki/Video_game_writer
/wiki/Game_director
/wiki/Video_game_producer
/wiki/Metal_Gear
/wiki/Snatcher_(video_game)
/wiki/Policenauts
/wiki/Zone_of_the_Enders
/wiki/Death_Stranding
/wiki/File:Hideo_Kojima_Signature.svg
/wiki/Video_game_designer
/wiki/Game_producer
/wiki/Auteur
/wiki/Video_games
#cite_note-2
/wiki/Action-adventure_film
/wiki/Literature
/wiki/Konami
/wiki/Metal_Gear_(video_game)
/wiki/MSX
/wiki/Stealth_game
/wiki/Metal_Gear
/wiki/Metal_Gear_Solid_(1998_video_game)
/wiki/PlayStation_(console)
/wiki/Zone_of_the_Enders
/wiki/Snatcher_(video_game)
/wiki/Policenauts
/wiki/Graphic_adventure_game
/wiki/Kojima_Productions
#cite_note-ign_bio-1
#cite_note-konami_pr-3
/wiki/Software_house
/wiki/Konami_Digital_Entertainment
#cite_note-promo_2011-4
/wiki/Death_Stranding
#cite_note-5
/wiki/Rolling_Stone
/wiki/Col

In [27]:
def get_links(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    links = []
    for link in soup.find_all('a'):
        links.append(link.get('href'))
    print(links)
get_links('http://google.com')

['http://www.google.com/imghp?hl=en&tab=wi', 'http://maps.google.com/maps?hl=en&tab=wl', 'https://play.google.com/?hl=en&tab=w8', 'http://www.youtube.com/?gl=US&tab=w1', 'https://news.google.com/?tab=wn', 'https://mail.google.com/mail/?tab=wm', 'https://drive.google.com/?tab=wo', 'https://www.google.com/intl/en/about/products?tab=wh', 'http://www.google.com/history/optout?hl=en', '/preferences?hl=en', 'https://accounts.google.com/ServiceLogin?hl=en&passive=true&continue=http://www.google.com/&ec=GAZAAQ', '/advanced_search?hl=en&authuser=0', '/intl/en/ads/', '/services/', '/intl/en/about.html', '/intl/en/policies/privacy/', '/intl/en/policies/terms/']
